In [42]:
import sys
import os

dir_path = os.path.abspath("..")

if dir_path not in sys.path:
    sys.path.append(dir_path)

#sys.path.append(os.path.dirname(os.path.realpath(__file__)) + "/..")

import bar_chart_race as bcr
import pandas as pd
from config import config
from enum import Enum
import mysql.connector
import datetime
import asyncio
import openskill
import trueskill
from dataclasses import dataclass
from typing import Union, Optional
import warnings
warnings.filterwarnings("ignore")

# dumbed down version of the laserforce_ranking 

class Team(Enum):
    RED = "red"
    GREEN = "green"
    BLUE = "blue"

class GameType(Enum):
    SM5 = "sm5"
    LASERBALL = "laserball"
    
class Role(Enum):
    SCOUT = "scout"
    HEAVY = "heavy"
    COMMANDER = "commander"
    MEDIC = "medic"
    AMMO = "ammo"

@dataclass
class SM5GamePlayer:
    player_id: int
    game_id: int
    team: Team
    role: Role
    score: int

@dataclass
class Player:
    id: int
    player_id: str
    ipl_id: str
    codename: str
    sm5_mu: float
    sm5_sigma: float
    laserball_mu: float
    laserball_sigma: float
    goals: int = 0
    assists: int = 0
    steals: int = 0
    clears: int = 0
    blocks: int = 0
    timestamp: datetime.datetime = datetime.datetime.now()
    game_player: Optional[SM5GamePlayer] = None
    
    @property
    def sm5_ordinal(self):
        return self.sm5_mu - 3 * self.sm5_sigma
    
    @property
    def laserball_ordinal(self):
        return self.laserball_mu - 3 * self.laserball_sigma
    
    @property
    def sm5_rating(self):
        return openskill.Rating(self.sm5_mu, self.sm5_sigma)
    
    @property
    def laserball_rating(self):
        return openskill.Rating(self.laserball_mu, self.laserball_sigma)
    
    @classmethod
    async def from_id(cls, id: int):
        cursor = mydb.cursor()
        cursor.execute("SELECT * FROM players WHERE id = %s", (id,))
        data = cursor.fetchone()
        if not data:
            return None
        ret = cls(*data)
        return ret

    @classmethod
    async def from_player_id(cls, player_id: str):
        cursor = mydb.cursor()
        cursor.execute("SELECT * FROM players WHERE player_id = %s", (player_id,))
        data = cursor.fetchone()
        if not data:
            return None
        ret = cls(*data)
        return ret
    
    @classmethod
    async def from_name(cls, name: str):
        cursor = mydb.cursor()
        cursor.execute("SELECT * FROM players WHERE codename = %s", (name,))
        data = cursor.fetchone()
        if not data:
            return None
        ret = cls(*data)
        return ret

@dataclass
class Game:
    id: int
    winner: Team
    type: GameType
    tdf: str = ""
    timestamp: datetime.datetime = datetime.datetime.now()
    players = []
    green = []
    red = []
    blue = []
    red_score = 0
    green_score = 0
    blue_score = 0

    @property
    def total_score(self):
        return self.red_score + self.green_score + self.blue_score

    async def _set_game_players(self):
        self.red = await self._get_game_players_team(Team.RED)

        if self.type == GameType.SM5:
            self.green = await self._get_game_players_team(Team.GREEN)
            self.players = [*self.red, *self.green]
        elif self.type == GameType.LASERBALL:
            self.blue = await self._get_game_players_team(Team.BLUE)
            self.players = [*self.red, *self.blue]

    async def _get_game_players_team(self, team: Team):
        cursor = mydb.cursor()
        cursor.execute(f"SELECT * FROM {self.type.value}_game_players WHERE `game_id` = %s AND `team` = %s", (self.id, team.value),)
        q = cursor.fetchall()
        final = []

        if self.type == GameType.SM5:
            for game_player in q:
                if game_player[0] == "":
                    player = Player(0, "", "", "", 25, 8.333, 25, 8.333)
                else:
                    player = await Player.from_player_id(game_player[0])

                if player:
                    player.game_player = SM5GamePlayer(game_player[0], game_player[1], Team(game_player[2]),
                                                   Role(game_player[3]), game_player[4])

                    final.append(player)

            if Team(game_player[2]) == Team.RED:
                self.red_score += game_player[4]
            elif Team(game_player[2]) == Team.GREEN:
                self.green_score += game_player[4]
            elif Team(game_player[2]) == Team.BLUE:
                self.blue_score += game_player[4]
        return final

mydb = mysql.connector.connect(
    host=config["db_host"],
    user=config["db_user"],
    password=config["db_password"],
    port=config["db_port"],
    database=config["db_database"]
)

async def get_players():
    cursor = mydb.cursor()
    cursor.execute("SELECT codename FROM players")
    q = cursor.fetchall()
    final = []
    for player in q:
        final.append(player[0])
    return final

data = {}
all_players = []

for player in await get_players():
    all_players.append(player)
    data[player] = []

i = 0

def attrgetter(obj, func):
    ret = []
    for i in obj:
        if callable(func):
            ret.append(func(i))
        elif isinstance(func, int):
            ret.append(i[func])
        else:
            ret.append(getattr(i, func))
    return ret

async def update_elo(game: Game, mode: GameType):
    mode = mode.value.lower()
    
    winner = game.winner
    
    team1 = game.red
    team1_mu = attrgetter(game.red, f"{mode}_mu")
    team1_sigma = attrgetter(game.red, f"{mode}_sigma")
    
    if mode == "sm5":
        team2 = game.green
        team2_mu = attrgetter(game.green, f"{mode}_mu")
        team2_sigma = attrgetter(game.green, f"{mode}_sigma")
    else: # laserball
        team2 = game.blue
        team2_mu = attrgetter(game.blue, f"{mode}_mu")
        team2_sigma = attrgetter(game.blue, f"{mode}_sigma")
    
    # convert to Rating
    team1_rating = []
    for i in range(len(team1)):
        team1_rating.append(openskill.Rating(team1_mu[i], team1_sigma[i]))
    
    team2_rating = []
    for i in range(len(team2)):
        team2_rating.append(openskill.Rating(team2_mu[i], team2_sigma[i]))

    if winner == Team.RED:  # red won
        team1_rating, team2_rating = openskill.rate([team1_rating, team2_rating])
    else:  # green/blue won
        team2_rating, team1_rating = openskill.rate([team2_rating, team1_rating])

    # convert back to Player 
    for i, p in enumerate(team1):
        setattr(p, f"{mode}_mu", team1_rating[i].mu)
        setattr(p, f"{mode}_sigma", team1_rating[i].sigma)
        
    for i, p in enumerate(team2):
        setattr(p, f"{mode}_mu", team2_rating[i].mu)
        setattr(p, f"{mode}_sigma", team2_rating[i].sigma)

    return (team1, team2)

async def update_elo_ts(game: Game, mode: GameType):
    mode = mode.value.lower()
    
    winner = game.winner
    
    team1 = game.red
    team1_mu = attrgetter(game.red, f"{mode}_mu")
    team1_sigma = attrgetter(game.red, f"{mode}_sigma")
    
    if mode == "sm5":
        team2 = game.green
        team2_mu = attrgetter(game.green, f"{mode}_mu")
        team2_sigma = attrgetter(game.green, f"{mode}_sigma")
    else: # laserball
        team2 = game.blue
        team2_mu = attrgetter(game.blue, f"{mode}_mu")
        team2_sigma = attrgetter(game.blue, f"{mode}_sigma")
    
    # convert to Rating
    team1_rating = []
    team1_weights = []
    for i in range(len(team1)):
        team1_rating.append(trueskill.Rating(team1_mu[i], team1_sigma[i]))
        team1_weights.append(game.red_score/team1[i].game_player.score)
    
    team2_rating = []
    team2_weights = []
    for i in range(len(team2)):
        team2_rating.append(trueskill.Rating(team2_mu[i], team2_sigma[i]))
        team2_weights.append(game.green_score/team2[i].game_player.score)

    print(team1_rating, team2_rating)

    if winner == Team.RED:  # red won
        team1_rating, team2_rating = trueskill.rate([team1_rating, team2_rating], weights=[team1_weights, team2_weights])
    else:  # green/blue won
        team2_rating, team1_rating = trueskill.rate([team2_rating, team1_rating], weights=[team2_weights, team1_weights])

    print(team1_rating, team2_rating)

    # convert back to Player 
    for i, p in enumerate(team1):
        setattr(p, f"{mode}_mu", team1_rating[i].mu)
        setattr(p, f"{mode}_sigma", team1_rating[i].sigma)
        
    for i, p in enumerate(team2):
        setattr(p, f"{mode}_mu", team2_rating[i].mu)
        setattr(p, f"{mode}_sigma", team2_rating[i].sigma)

    return (team1, team2)

async def update_game_elo(game):
    global df
    # gets id of the inserted game
    
    game.red, game.green = await update_elo_ts(game, GameType.SM5)
    game.players = [*game.red, *game.green]

    # update openskill

    players = all_players.copy()
    
    for player in game.players:
        if player.codename == "":
            continue # not logged in player
        players.remove(player.codename)
        data[player.codename].append(player.sm5_mu - 3 * player.sm5_sigma)

    for player in players:
        if len(data[player]) == 0:
            data[player].append(0)
        else:
            data[player].append(data[player][-1])

async def get_all_games():
    games = []
    cursor = mydb.cursor()
    cursor.execute("SELECT COUNT(*) FROM games")
    count = cursor.fetchone()
    game_count = count[0]

    for id in range(1, game_count + 50):
        cursor.execute("SELECT * FROM games WHERE id = %s", (id,))
        data = cursor.fetchone()
        
        if not data:
            continue
        game = Game(*data)

        game.type = GameType(game.type)
        game.winner = Team(game.winner)

        await game._set_game_players()
        if game:
            games.append(game)
    
    return games

In [43]:
# update elo and format data
games = await get_all_games()
i = 0

for game in games:
    if game.type == GameType.SM5:
        await update_game_elo(game)
    i += 1

data = {key:val for key, val in data.items() if val.count(0) != len(val)}

df = pd.DataFrame(data=data)

[trueskill.Rating(mu=27.548, sigma=6.476), trueskill.Rating(mu=26.160, sigma=6.505), trueskill.Rating(mu=22.677, sigma=6.733), trueskill.Rating(mu=25.312, sigma=6.681), trueskill.Rating(mu=23.303, sigma=8.240), trueskill.Rating(mu=22.643, sigma=7.119)] [trueskill.Rating(mu=30.015, sigma=5.973), trueskill.Rating(mu=23.104, sigma=6.391), trueskill.Rating(mu=20.841, sigma=6.661), trueskill.Rating(mu=28.299, sigma=7.769), trueskill.Rating(mu=22.414, sigma=6.859), trueskill.Rating(mu=17.673, sigma=6.056), trueskill.Rating(mu=22.135, sigma=6.334)]
(trueskill.Rating(mu=27.548, sigma=6.477), trueskill.Rating(mu=26.160, sigma=6.505), trueskill.Rating(mu=22.677, sigma=6.733), trueskill.Rating(mu=25.312, sigma=6.682), trueskill.Rating(mu=23.303, sigma=8.241), trueskill.Rating(mu=22.643, sigma=7.120)) (trueskill.Rating(mu=30.015, sigma=5.974), trueskill.Rating(mu=23.104, sigma=6.391), trueskill.Rating(mu=20.841, sigma=6.661), trueskill.Rating(mu=28.299, sigma=7.770), trueskill.Rating(mu=22.414, si

In [44]:
# show graph
bcr.bar_chart_race(df=df, filename=None, title="SM5 players at ILT skill rating over time (Weighted by score)", n_bars=20, period_fmt="Game {x:.0f}")